<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 550.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Algoritme: SSL Regression Heat Diffusion on graphs

In [5]:
from scipy import sparse
#from gbssl import LGC,HMN,PARW,OMNI,CAMLP
import random
import numpy as np
from sklearn.neighbors import kneighbors_graph

from sklearn.preprocessing import normalize
from scipy.io import loadmat

from sklearn.metrics.pairwise import rbf_kernel
from scipy.sparse import lil_matrix
from scipy import linalg
from sklearn.metrics import log_loss

from sklearn.neighbors import KNeighborsRegressor
from scipy.sparse.linalg import spsolve
from scipy.sparse import identity
import math
import warnings
warnings.filterwarnings("ignore")

In [6]:
def get_graph_matrix(X, k):

    # KNN graph
    KNN_graf = kneighbors_graph(X,
                                n_neighbors=k,
                                mode ="connectivity",
                                include_self=False)

    # Sigma
    n_egenskaper = X.shape[1]
    sigma = 1.0/ n_egenskaper

    # Normalisering for å unnga bias
    #distanser = np.sqrt((KNN_graf ** 2).sum(axis=1))
    #normalisert_KNN_graf = KNN_graf / distanser[: , np.newaxis]

    # RBF funksjon for vekter
    RBF_vekter = rbf_kernel(X, X, gamma=1.0 / (2 * sigma ** 2))

    KNN_RBF_graf = sparse.csr_matrix.multiply(KNN_graf, RBF_vekter)

    return KNN_RBF_graf

def get_harmonic_scores(G,treningsnode_markert_id, treningsnode_umarkert_id,testnode_id,yl):

    ##create Propagation Matrix###

    n = G.shape[0]
    graph = G.copy()
    degrees = graph.sum(axis=0).A[0]
    degrees[degrees==0] += 1  # Avoid division by 0
    D = sparse.diags((1.0/degrees),offsets=0)
    P = D.dot(graph).tolil() # Transition matrix
    P[treningsnode_markert_id] = 0
    all_values = np.zeros(n)
    all_values[treningsnode_markert_id] = yl

    all_values[treningsnode_umarkert_id] = 0
    all_values[testnode_id] = 0
    state_vector = all_values.copy()


    ###Propagate the scores####
    remaining_iter = 30
    state = state_vector.copy()
    Base = state.copy() # Initialize U = Y
    P = P.A

    while remaining_iter > 0:
        state = P.dot(state) + Base # Y^(k+1) <- Y^(k+1) + U
        remaining_iter -= 1

    state = np.round_(state, decimals=2)

    return state


def bhd(X_data, y_data, treningsnoder_markert, treningsnoder_umarkert, testnoder, k, alpha=1.0, maks_iterasjoner=100):

  graph_data = get_graph_matrix(X_data, k)
  y_trening_markert = y_data[treningsnoder_markert]

  state_hmn = get_harmonic_scores(graph_data,treningsnoder_markert, treningsnoder_umarkert, testnoder, y_trening_markert)

  graph = graph_data.copy()
  n = graph.shape[0]
  all_values = y_data.copy() # label vector
  all_values[testnoder] = np.mean(all_values[treningsnoder_markert]) #We imputed the values for the inlabeled nodes as the means of the labeled nodes
  all_values[treningsnoder_umarkert] = np.mean(all_values[treningsnoder_markert])
  remaining_iter = maks_iterasjoner

  I = np.eye(n,n,dtype=np.float64) # Identity matrix
  C = all_values - state_hmn # Constant C
  state = C.copy() # state vector
  state = state.reshape(n,1)
  L = sparse.csgraph.laplacian(graph,normed=True) # Laplacian matrix
  V = I + (-alpha/remaining_iter) * L

  while remaining_iter > 0:
      state = V.dot(state)
      remaining_iter -= 1

  state = state_hmn.reshape(n,1) + state

  y_prediksjon = state

  y_trening_prediksjon = np.ravel(y_prediksjon[treningsnoder_markert])
  y_test_prediksjon = np.ravel(y_prediksjon[testnoder])

  return y_test_prediksjon, y_trening_prediksjon

# Egendefinerte moduler

In [7]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [8]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import umarkert_treningssett_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag

In [9]:
def evaluering_beregning(y_test, y_test_prediksjon, y_trening, y_trening_prediksjon):
  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)

  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [10]:
def rmse_validering_StratifiedKFold(modell, datasett, respons, n_splitt):

  skf = StratifiedKFold(n_splits=n_splitt)

  rmse_validering_resultat = []

  for trening, validering in skf.split(datasett, respons):
    X_trening_fold = (datasett.iloc[trening, :]).iloc[:, :-1]
    X_validering_fold = (datasett.iloc[validering, :]).iloc[:, :-1]

    y_trening_fold = (datasett.iloc[trening, :]).iloc[:, -1]
    y_validering_fold = (datasett.iloc[validering, :]).iloc[:, -1]

    modell.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = modell.predict(X_validering_fold)

    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  return mean(rmse_validering)

# Importering av relevant data

In [11]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

collagen_data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

collagen_data_design.index = pd.to_datetime(collagen_data_design.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [12]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [13]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatPercent  29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT08           29136 non-null  float64
 11  TT20           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [14]:
collagen_data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18568 entries, 2022-11-02 00:00:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18568 non-null  int64  
 1   EnzymeType_A2  18568 non-null  int64  
 2   EnzymeType_B   18568 non-null  int64  
 3   EnzymeType_C   18568 non-null  int64  
 4   EnzymeType_D   18568 non-null  int64  
 5   EnzymeType_E   18568 non-null  int64  
 6   RawMatPercent  18568 non-null  float64
 7   NIRfat         18568 non-null  float64
 8   NIRash         18568 non-null  float64
 9   NIRwater       18568 non-null  float64
 10  TT08           18568 non-null  float64
 11  TT20           18568 non-null  float64
 12  TT12           18568 non-null  float64
 13  Collagen       31 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [15]:
collagen_data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10568 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10568 non-null  int64  
 1   EnzymeType_A2  10568 non-null  int64  
 2   EnzymeType_B   10568 non-null  int64  
 3   EnzymeType_C   10568 non-null  int64  
 4   EnzymeType_D   10568 non-null  int64  
 5   EnzymeType_E   10568 non-null  int64  
 6   RawMatPercent  10568 non-null  float64
 7   NIRfat         10568 non-null  float64
 8   NIRash         10568 non-null  float64
 9   NIRwater       10568 non-null  float64
 10  TT08           10568 non-null  float64
 11  TT20           10568 non-null  float64
 12  TT12           10568 non-null  float64
 13  Collagen       58 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Moduler for optimalisering etter treningsandel etter algoritme

## Modelltrening etter enzymtype

### CoReg

In [16]:
def trening_coreg_enzym(treningssett_markert,
                        enzymtyper_treningssett_markert,
                        treningssett_umarkert,
                        treningssett,
                        X_test,
                        y_test,
                        splitt,
                        gjentagelser,
                        n_runder,
                        trenings_andel=1.0):



    X_trening = treningssett.iloc[:, :-1]
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    skalerer = StandardScaler()
    skalerer.fit(X_trening)
    X_trening_markert_transformert = skalerer.transform(X_trening_markert)
    X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
    X_test_transformert = skalerer.transform(X_test)

    #CoReg
    def objective(trial):

      parametere = {
          'k1': trial.suggest_int('k1', 1, 10),
          'k2': trial.suggest_int('k2', 1, 10),
          'p1': trial.suggest_int('p1', 1, 10),
          'p2': trial.suggest_int('p2', 1, 10),
          'max_iters': trial.suggest_int('max_iters', 1, 20),
          'pool_size': trial.suggest_int('pool_size', 10, 200)
      }


      coreg = CoReg(**parametere)

      rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                    n_repeats=gjentagelser,
                                    random_state=random_seed)

      rmse_validering_resultat = []

      for train, test in rskf.split(treningssett_markert,
                                    enzymtyper_treningssett_markert):

        # Markerte folds
        trenings_fold_markert = treningssett_markert.iloc[train, :]
        validerings_fold = treningssett_markert.iloc[test, :]

        # Markert respons
        y_trening_fold = trenings_fold_markert.iloc[:, -1]
        y_validering_fold = validerings_fold.iloc[:, -1]


        #Fullstendig treningsdata, markert og umarkert folds
        X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
        X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert], axis=0)

        # Skalerer folds etter treningsset fold, inkl. markert og umarkert
        skalerer_fold = StandardScaler()
        skalerer_fold.fit(X_trening_fold)

        X_trening_fold_markert_transformert = skalerer_fold.transform(X_trening_fold_markert)
        X_trening_fold_umarkert_transformert = skalerer_fold.transform(X_trening_umarkert)

        X_validering_fold = validerings_fold.iloc[:, :-1]
        X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)


        coreg.fit(X_trening_fold_markert_transformert,
                  y_trening_fold,
                  X_trening_fold_umarkert_transformert)

        y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
        rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt

    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)


    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    coreg = CoReg(**beste_parametere)


    # Trener modell for videre evaluering ´
    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)


    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test, mae_test, mape_test,\
    rmse_trening, r2_trening, mae_trening, mape_trening\
    = evaluering_beregning(y_test,
                            y_test_prediksjon,
                            y_trening_markert,
                            y_trening_prediksjon)


    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]

    return resultater

### BHD

In [17]:
def trening_bhd_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      treningssett_umarkert,
                      enzymtyper_treningssett_umarkert,
                      treningssett,
                      testsett,
                      X_test,
                      y_test,
                      splitt,
                      gjentagelser,
                      n_runder,
                      umarkert_andel=0.25,
                      trenings_andel=1.0):


    # Markert data
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    # Umarkert data
    #X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]

    # Velger andel umarkert data som skal inkluderes
    #umarkert_rader = int(np.ceil(X_trening_umarkert.shape[0]*umarkert_andel))
    #X_trening_umarkert_filtrert = X_trening_umarkert.iloc[:umarkert_rader, :]

    # Umarkert data, andel
    ubenyttet_andel = (1.0 - umarkert_andel)
    treningssett_umarkert_filtrert, _, _, _= train_test_split(treningssett_umarkert,
                                                          enzymtyper_treningssett_umarkert,
                                                          test_size=ubenyttet_andel,
                                                          stratify=enzymtyper_treningssett_umarkert,
                                                          random_state=random_seed)

    X_trening_umarkert_filtrert = treningssett_umarkert_filtrert.iloc[:, :-1]

    # Trenigsdata, markert og filtrert umarkert
    X_trening_filtrert = pd.concat([X_trening_markert, X_trening_umarkert_filtrert], axis=0)
    X_trening_filtrert = X_trening_filtrert.sort_index(ascending=True)

    # Skalerer data etter filtrert treningsdata, inkl. markert og umarkert
    skalerer = StandardScaler()
    skalerer.fit(X_trening_filtrert)
    X_trening_markert_transformert = X_trening_markert.copy()
    X_trening_markert_transformert.iloc[:, :] = skalerer.transform(X_trening_markert)

    X_trening_umarkert_transformert = X_trening_umarkert_filtrert.copy()
    X_trening_umarkert_transformert.iloc[:, :] = skalerer.transform(X_trening_umarkert_filtrert)

    X_test_transformert = X_test.copy()
    X_test_transformert.iloc[:,:] = skalerer.transform(X_test)

    # Parameter tuning
    def objective(trial):

      parametere = {
          'k': trial.suggest_int('k', 1000, 7000),
          'alpha': trial.suggest_categorical('alpha', [0.01, 0.1, 1, 10]),
          'maks_iterasjoner': trial.suggest_int('maks_iterasjoner', 1, 200),
      }

      # Kryss-validering
      rskf = RepeatedStratifiedKFold(n_splits=splitt,
                                    n_repeats=gjentagelser,
                                    random_state=random_seed)

      rmse_validering_resultat = []

      for train, test in rskf.split(treningssett_markert,
                                    enzymtyper_treningssett_markert):

        # Markerte folds
        trenings_fold_markert = treningssett_markert.iloc[train, :]
        validerings_fold = treningssett_markert.iloc[test, :]

        # Markert respons
        y_trening_fold = trenings_fold_markert.iloc[:, -1]
        y_validering_fold = validerings_fold.iloc[:, -1]

        #Fullstendig treningsdata, markert og umarkert folds
        X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
        X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert_filtrert], axis=0)

        #Skalerer folds etter treningssett fold, inkl. markert og umarkert
        skalerer_fold = StandardScaler()
        skalerer_fold.fit(X_trening_fold)

        X_trening_fold_markert_transformert = X_trening_fold_markert.copy()
        X_trening_fold_markert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_fold_markert)

        X_trening_fold_umarkert_filtrert_transformert = X_trening_umarkert_filtrert.copy()
        X_trening_fold_umarkert_filtrert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_umarkert_filtrert)

        X_validering_fold = validerings_fold.iloc[:, :-1]
        X_validering_fold_transformert = X_validering_fold.copy()
        X_validering_fold_transformert.iloc[:, :] = skalerer_fold.transform(X_validering_fold)


        # Samler data som skal inkluderes etter valgt andel umarkert data, skalert forklaringsvariabler
        X_data_filtrert = pd.concat([X_trening_fold_markert_transformert, X_trening_fold_umarkert_filtrert_transformert], axis=0)
        X_data_filtrert = pd.concat([X_data_filtrert, X_validering_fold_transformert], axis=0)
        X_data_filtrert = X_data_filtrert.sort_index(ascending=True)


        # Samler data som skal inkluderes etter valgt andel umarkert data, responsvariabler
        data_filtrert = pd.concat([trenings_fold_markert, treningssett_umarkert_filtrert], axis=0)
        data_filtrert = pd.concat([data_filtrert, validerings_fold], axis=0)
        data_filtrert = data_filtrert.sort_index(ascending=True)

        # Inspirert fra Script
        X_verdi = X_data_filtrert.values
        n = X_verdi.shape[0]
        y_verdi = data_filtrert.iloc[:, -1].values

        x = np.arange(n)

        index_trening_markert = np.nonzero(data_filtrert.index.isin(trenings_fold_markert.index))[0]
        index_trening_umarkert = np.nonzero(data_filtrert.index.isin(treningssett_umarkert_filtrert.index))[0]
        index_validering = np.nonzero(data_filtrert.index.isin(validerings_fold.index))[0]

        trening_noder_markert = x[index_trening_markert]
        trening_noder_umarkert = x[index_trening_umarkert]
        validering_noder = x[index_validering]

        y_validering_prediksjon, y_trening_prediksjon = bhd(X_data = X_verdi,
                                                            y_data = y_verdi,
                                                            treningsnoder_markert = trening_noder_markert,
                                                            treningsnoder_umarkert = trening_noder_umarkert,
                                                            testnoder = validering_noder,
                                                            **parametere)

        y_validering, y_trening = y_verdi[index_validering], y_verdi[index_trening_markert]

        rmse_validering = rmse(y_validering, y_validering_prediksjon)

        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt

    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)


    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    # Samler data som skal inkluderes etter valgt andel umarkert data, skalert forklaringsvariabler
    X_data_filtrert = pd.concat([X_trening_markert_transformert, X_trening_umarkert_transformert], axis=0)
    X_data_filtrert = pd.concat([X_data_filtrert, X_test_transformert], axis=0)
    X_data_filtrert = X_data_filtrert.sort_index(ascending=True)


    # Samler data som skal inkluderes etter valgt andel umarkert data, responsvariabler
    data_filtrert = pd.concat([treningssett_markert, treningssett_umarkert_filtrert], axis=0)
    data_filtrert = pd.concat([data_filtrert, testsett], axis=0)
    data_filtrert = data_filtrert.sort_index(ascending=True)

    # Inspirtert av Script
    X_verdi = X_data_filtrert.values
    n = X_verdi.shape[0]
    y_verdi = data_filtrert.iloc[:, -1].values

    x = np.arange(n)

    index_trening_markert = np.nonzero(data_filtrert.index.isin(treningssett_markert.index))[0]
    index_trening_umarkert = np.nonzero(data_filtrert.index.isin(treningssett_umarkert_filtrert.index))[0]
    index_test = np.nonzero(data_filtrert.index.isin(testsett.index))[0]

    trening_noder_markert = x[index_trening_markert]
    trening_noder_umarkert = x[index_trening_umarkert]
    test_noder = x[index_test]

    y_test_prediksjon, y_trening_prediksjon = bhd(X_data = X_verdi,
                                                  y_data = y_verdi,
                                                  treningsnoder_markert = trening_noder_markert,
                                                  treningsnoder_umarkert = trening_noder_umarkert,
                                                  testnoder = test_noder,
                                                  **beste_parametere)

    y_test, y_trening = y_verdi[index_test], y_verdi[index_trening_markert]

    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test, mae_test, mape_test,\
    rmse_trening, r2_trening, mae_trening, mape_trening\
    = evaluering_beregning(y_test,
                            y_test_prediksjon,
                            y_trening,
                            y_trening_prediksjon)


    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]

    return resultater

## Modelltrening etter dag og kontinuitet

### CoReg

In [22]:
def trening_coreg_dag(treningssett_markert,
                      treningssett_umarkert,
                      treningssett,
                      X_test,
                      y_test,
                      splitt,
                      n_runder,
                      trenings_andel=1.0):


    X_trening = treningssett.iloc[:, :-1]
    X_trening_markert = treningssett_markert.iloc[:, :-1]
    X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
    y_trening_markert = treningssett_markert.iloc[:, -1]

    skalerer = StandardScaler()
    skalerer.fit(X_trening)
    X_trening_markert_transformert = skalerer.transform(X_trening_markert)
    X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
    X_test_transformert = skalerer.transform(X_test)

    def objective(trial):

      parametere = {
          'k1': trial.suggest_int('k1', 1, 10),
          'k2': trial.suggest_int('k2', 1, 10),
          'p1': trial.suggest_int('p1', 1, 10),
          'p2': trial.suggest_int('p2', 1, 10),
          'max_iters': trial.suggest_int('max_iters', 1, 20),
          'pool_size': trial.suggest_int('pool_size', 10, 200)
      }


      coreg = CoReg(**parametere)

      treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      treningssett_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
      rmse_validering_resultat = []

      for i, j in zip(treningssett, valideringssett):
        train = treningssett[i]
        val = valideringssett[j]

        # Markert folds
        trenings_fold_markert = trening_markert.loc[train, :]
        validerings_fold = trening_markert.loc[val, :]

        # Markert respons
        y_trening_fold = trenings_fold_markert.iloc[:, -1]
        y_validerings_fold = validerings_fold.iloc[:, -1]

        #Fullstendig treningsdata, markert og umarkert folds
        X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
        X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert], axis=0)

        # Skalerer folds etter treningsset fold, inkl. markert og umarkert
        skalerer_fold = StandardScaler()
        skalerer_fold.fit(X_trening_fold)

        X_trening_fold_markert_transformert = skalerer_fold.transform(X_trening_fold_markert)
        X_trening_fold_umarkert_transformert = skalerer_fold.transform(X_trening_umarkert)

        X_validering_fold = validerings_fold.iloc[:, :-1]
        X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)


        coreg.fit(X_trening_fold_markert_transformert,
                  y_trening_fold,
                  X_trening_fold_umarkert_transformert)

        y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
        rmse_validering = rmse(y_validerings_fold, y_validering_prediksjon)
        rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

      return rmse_validering_snitt


    if __name__ == "__main__":
      study = optuna.create_study(direction='minimize')
      study.optimize(objective, n_trials=n_runder)


    # Bruker beste paramtere for studiet for å lage modell
    beste_parametere = study.best_params

    coreg = CoReg(**beste_parametere)


    # Trener modell for videre evaluering ´
    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)


    # Henter ut evalueringsresulateter av beste modell fra undersøkelse
    rmse_test, r2_test, mae_test, mape_test,\
    rmse_trening, r2_trening, mae_trening, mape_trening\
    = evaluering_beregning(y_test,
                            y_test_prediksjon,
                            y_trening_markert,
                            y_trening_prediksjon)


    # Legger resulater til i datasett
    resultater = [trenings_andel,
                  rmse_test, r2_test, mae_test, mape_test,
                  rmse_trening, r2_trening, mae_trening, mape_trening,
                  beste_parametere]

    return resultater

### BHD

In [23]:
def trening_bhd_dag(treningssett_markert,
                    treningssett_umarkert,
                    treningssett,
                    X_test,
                    y_test,
                    splitt,
                    n_runder,
                    umarkert_andel=0.25,
                    trenings_andel=1.0):

  # Markert data
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Umarkert data
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]

  # Velger andel umarkert data som skal inkluderes
  umarkert_andel = 0.25
  umarkert_rader = int(np.ceil(X_trening_umarkert.shape[0]*umarkert_andel))
  X_trening_umarkert_filtrert = X_trening_umarkert.iloc[:umarkert_rader, :]

  # Trenigsdata, markert og filtrert umarkert
  X_trening_filtrert = pd.concat([X_trening_markert, X_trening_umarkert_filtrert], axis=0)
  X_trening_filtrert = X_trening_filtrert.sort_index(ascending=True)

  # Skalerer data etter filtrert treningsdata, inkl. markert og umarkert
  skalerer = StandardScaler()
  skalerer.fit(X_trening_filtrert)
  X_trening_markert_transformert = X_trening_markert.copy()
  X_trening_markert_transformert.iloc[:, :] = skalerer.transform(X_trening_markert)

  X_trening_umarkert_transformert = X_trening_umarkert.copy()
  X_trening_umarkert_transformert.iloc[:, :] = skalerer.transform(X_trening_umarkert)

  X_test_transformert = X_test.copy()
  X_test_transformert.iloc[:,:] = skalerer.transform(X_test)

  # Parameter tuning
  def objective(trial):

    parametere = {
          'k': trial.suggest_int('k', 1000, 7000),
          'alpha': trial.suggest_categorical('alpha', [0.01, 0.1, 1, 10]),
          'maks_iterasjoner': trial.suggest_int('maks_iterasjoner', 1, 200),
      }

    # Kryss-validering

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                    treningssett_markert,
                                                                    n_splitt=splitt,
                                                                    trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      # Markerte folds
      trenings_fold_markert = treningssett_markert.loc[train, :]
      validerings_fold = treningssett_markert.loc[val, :]

      # Markert respons
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_validering_fold = validerings_fold.iloc[:, -1]

      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert_filtrert], axis=0)

      #Skalerer folds etter treningssett fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_markert_transformert = X_trening_fold_markert.copy()
      X_trening_fold_markert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_fold_markert)

      X_trening_fold_umarkert_filtrert_transformert = X_trening_umarkert_filtrert.copy()
      X_trening_fold_umarkert_filtrert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_umarkert_filtrert)

      X_validering_fold = validerings_fold.iloc[:, :-1]
      X_validering_fold_transformert = X_validering_fold.copy()
      X_validering_fold_transformert.iloc[:, :] = skalerer_fold.transform(X_validering_fold)


      # Samler data som skal inkluderes etter valgt andel umarkert data, skalert forklaringsvariabler
      X_data_filtrert = pd.concat([X_trening_fold_markert_transformert, X_trening_fold_umarkert_filtrert_transformert], axis=0)
      X_data_filtrert = pd.concat([X_data_filtrert, X_validering_fold_transformert], axis=0)
      X_data_filtrert = X_data_filtrert.sort_index(ascending=True)


      # Samler data som skal inkluderes etter valgt andel umarkert data, responsvariabler
      treningssett_umarkert_filtrert = treningssett_umarkert.iloc[:umarkert_rader, :]
      data_filtrert = pd.concat([trenings_fold_markert, treningssett_umarkert_filtrert], axis=0)
      data_filtrert = pd.concat([data_filtrert, validerings_fold], axis=0)
      data_filtrert = data_filtrert.sort_index(ascending=True)

      # Inspirert fra Script
      X_verdi = X_data_filtrert.values
      n = X_verdi.shape[0]
      y_verdi = data_filtrert.iloc[:, -1].values

      x = np.arange(n)

      index_trening_markert = np.nonzero(data_filtrert.index.isin(trenings_fold_markert.index))[0]
      index_trening_umarkert = np.nonzero(data_filtrert.index.isin(treningssett_umarkert_filtrert.index))[0]
      index_validering = np.nonzero(data_filtrert.index.isin(validerings_fold.index))[0]

      trening_noder_markert = x[index_trening_markert]
      trening_noder_umarkert = x[index_trening_umarkert]
      validering_noder = x[index_validering]

      y_validering_prediksjon, y_trening_prediksjon = bhd(X_data = X_verdi,
                                                          y_data = y_verdi,
                                                          treningsnoder_markert = trening_noder_markert,
                                                          treningsnoder_umarkert = trening_noder_umarkert,
                                                          testnoder = validering_noder,
                                                          **parametere)

      y_validering, y_trening = y_verdi[index_validering], y_verdi[index_trening_markert]

      rmse_validering = rmse(y_validering, y_validering_prediksjon)

      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_runder)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  # Samler data som skal inkluderes etter valgt andel umarkert data, skalert forklaringsvariabler
  X_data_filtrert = pd.concat([X_trening_markert_transformert, X_trening_umarkert_transformert], axis=0)
  X_data_filtrert = pd.concat([X_data_filtrert, X_test_transformert], axis=0)
  X_data_filtrert = X_data_filtrert.sort_index(ascending=True)


  # Samler data som skal inkluderes etter valgt andel umarkert data, responsvariabler
  trening_umarkert_filtrert = trening_umarkert.iloc[:umarkert_rader, :]
  data_filtrert = pd.concat([treningssett_markert, trening_umarkert_filtrert], axis=0)
  data_filtrert = pd.concat([data_filtrert, test], axis=0)
  data_filtrert = data_filtrert.sort_index(ascending=True)

  # Inspirtert av Script
  X_verdi = X_data_filtrert.values
  n = X_verdi.shape[0]
  y_verdi = data_filtrert.iloc[:, -1].values

  x = np.arange(n)

  index_trening_markert = np.nonzero(data_filtrert.index.isin(treningssett_markert.index))[0]
  index_trening_umarkert = np.nonzero(data_filtrert.index.isin(trening_umarkert_filtrert.index))[0]
  index_test = np.nonzero(data_filtrert.index.isin(test.index))[0]

  trening_noder_markert = x[index_trening_markert]
  trening_noder_umarkert = x[index_trening_umarkert]
  test_noder = x[index_test]

  y_test_prediksjon, y_trening_prediksjon = bhd(X_data = X_verdi,
                                                y_data = y_verdi,
                                                treningsnoder_markert = trening_noder_markert,
                                                treningsnoder_umarkert = trening_noder_umarkert,
                                                testnoder = test_noder,
                                                **beste_parametere)

  y_test, y_trening = y_verdi[index_test], y_verdi[index_trening_markert]

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                          y_test_prediksjon,
                          y_trening_markert,
                          y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  return resultater

## Lagring av resultater

In [24]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Trenings andel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening",
                       "Beste parametere"]


# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [28]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett, _,\
treningssett_umarkert, enzymtyper_treningssett_umarkert, treningssett\
=trening_testsett_oppdeling_enzym(rå_data,
                                  collagen_data,
                                  test_andel=test_andel)


# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 4

# Antall runder til optimalisering
n_runder = 20

### CoReg

In [26]:
# Modelloptimalisering
coreg_resultater = trening_coreg_enzym(treningssett_markert,
                                        enzymtyper_treningssett_markert,
                                        treningssett_umarkert,
                                        treningssett,
                                        X_test,
                                        y_test,
                                        splitt=splitt,
                                        gjentagelser=gjentagelser,
                                        n_runder=n_runder,
                                        trenings_andel=trenings_andel)


# Legger resulater til i datasett
coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-16 19:46:16,962] A new study created in memory with name: no-name-6cc5cb10-1349-42db-884a-50b5b1bb57d0
[I 2024-03-16 19:46:38,890] Trial 0 finished with value: 4.528656684269792 and parameters: {'k1': 6, 'k2': 5, 'p1': 8, 'p2': 9, 'max_iters': 4, 'pool_size': 112}. Best is trial 0 with value: 4.528656684269792.
[I 2024-03-16 19:48:50,293] Trial 1 finished with value: 4.390971076123619 and parameters: {'k1': 6, 'k2': 3, 'p1': 8, 'p2': 2, 'max_iters': 15, 'pool_size': 196}. Best is trial 1 with value: 4.390971076123619.
[I 2024-03-16 19:50:44,081] Trial 2 finished with value: 4.333299657215984 and parameters: {'k1': 10, 'k2': 6, 'p1': 10, 'p2': 10, 'max_iters': 20, 'pool_size': 79}. Best is trial 2 with value: 4.333299657215984.
[I 2024-03-16 19:50:53,125] Trial 3 finished with value: 4.605033764314296 and parameters: {'k1': 4, 'k2': 1, 'p1': 10, 'p2': 7, 'max_iters': 11, 'pool_size': 28}. Best is trial 2 with value: 4.333299657215984.
[I 2024-03-16 19:52:30,621] Trial 4 finis

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,3.82335,0.579357,3.305174,0.165158,1.754067,0.87858,1.429657,0.068071,"{'k1': 1, 'k2': 8, 'p1': 1, 'p2': 10, 'max_ite..."


### BHD

In [29]:
# BHD
bhd_resultater = trening_bhd_enzym(treningssett_markert,
                                   enzymtyper_treningssett_markert,
                                   treningssett_umarkert,
                                   enzymtyper_treningssett_umarkert,
                                   treningssett,
                                   testsett,
                                    X_test,
                                    y_test,
                                    splitt=splitt,
                                    gjentagelser=gjentagelser,
                                    n_runder=n_runder,
                                    umarkert_andel=umarkert_andel,
                                    trenings_andel=trenings_andel)


# Legger resulater til i datasett
bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-17 12:57:47,124] A new study created in memory with name: no-name-619d6be5-b4cd-4d62-966c-21599295372f
[I 2024-03-17 13:00:31,872] Trial 0 finished with value: 5.292751618978724 and parameters: {'k': 2338, 'alpha': 0.1, 'maks_iterasjoner': 46}. Best is trial 0 with value: 5.292751618978724.
[I 2024-03-17 13:04:30,331] Trial 1 finished with value: 13.388645094157992 and parameters: {'k': 4346, 'alpha': 10, 'maks_iterasjoner': 159}. Best is trial 0 with value: 5.292751618978724.
[I 2024-03-17 13:08:31,432] Trial 2 finished with value: 5.292521169227146 and parameters: {'k': 2861, 'alpha': 0.1, 'maks_iterasjoner': 193}. Best is trial 2 with value: 5.292521169227146.
[I 2024-03-17 13:12:49,527] Trial 3 finished with value: 5.173500319103635 and parameters: {'k': 4815, 'alpha': 0.01, 'maks_iterasjoner': 171}. Best is trial 3 with value: 5.173500319103635.
[I 2024-03-17 13:15:57,348] Trial 4 finished with value: 5.293381004423575 and parameters: {'k': 4807, 'alpha': 0.1, 'maks_ite

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,1.0,5.965666,-0.024102,4.49114,0.240441,0.049112,0.999905,0.037417,0.001832,"{'k': 6901, 'alpha': 0.01, 'maks_iterasjoner':..."


## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  coreg_resultater = trening_coreg_enzym(treningssett_markert,
                                         enzymtyper_treningssett_markert,
                                         treningssett_umarkert,
                                         treningssett,
                                         X_test,
                                         y_test,
                                         splitt=splitt,
                                         gjentagelser=gjentagelser,
                                         n_runder=n_runder,
                                         trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-15 16:24:07,402] A new study created in memory with name: no-name-0c2bb0a8-431e-4333-bf4d-caef6d218a7d
[I 2024-03-15 16:24:47,898] Trial 0 finished with value: 3.8063367883903254 and parameters: {'k1': 9, 'k2': 3, 'p1': 5, 'p2': 3, 'max_iters': 3, 'pool_size': 163}. Best is trial 0 with value: 3.8063367883903254.
[I 2024-03-15 16:25:51,558] Trial 1 finished with value: 3.8001781496954283 and parameters: {'k1': 8, 'k2': 2, 'p1': 7, 'p2': 4, 'max_iters': 10, 'pool_size': 126}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:26:01,396] Trial 2 finished with value: 3.8225169644550157 and parameters: {'k1': 9, 'k2': 3, 'p1': 6, 'p2': 3, 'max_iters': 9, 'pool_size': 18}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:28:23,118] Trial 3 finished with value: 3.84186936894447 and parameters: {'k1': 6, 'k2': 10, 'p1': 2, 'p2': 2, 'max_iters': 11, 'pool_size': 174}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:29:47,396] Trial 4 fi

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,3.806806,0.582990,3.097417,0.159482,2.503015,0.751529,2.052800,0.094294,"{'k1': 9, 'k2': 3, 'p1': 6, 'p2': 1, 'max_iter..."
1,1.0,3.832337,0.577377,3.174067,0.165273,2.124103,0.821947,1.727685,0.081852,"{'k1': 9, 'k2': 2, 'p1': 10, 'p2': 2, 'max_ite..."


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # BHD
  bhd_resultater = trening_bhd_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     umarkert_andel=umarkert_andel,
                                     trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-15 20:17:35,401] A new study created in memory with name: no-name-7213be66-d841-4aa5-ae4c-db1a343129e5
[I 2024-03-15 20:19:27,819] Trial 0 finished with value: 5.246828854906072 and parameters: {'k': 5046, 'alpha': 0.1, 'maks_iterasjoner': 79}. Best is trial 0 with value: 5.246828854906072.
[I 2024-03-15 20:21:01,095] Trial 1 finished with value: 5.246813682909695 and parameters: {'k': 2726, 'alpha': 0.1, 'maks_iterasjoner': 99}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:22:19,322] Trial 2 finished with value: 5.246898540250276 and parameters: {'k': 2368, 'alpha': 0.1, 'maks_iterasjoner': 41}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:24:09,995] Trial 3 finished with value: 8.236193845576382 and parameters: {'k': 4668, 'alpha': 1, 'maks_iterasjoner': 139}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:25:09,395] Trial 4 finished with value: 8.243883651545284 and parameters: {'k': 1153, 'alpha': 1, 'maks_iterasjone

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 collagen_data,
                                 test_andel=test_andel,
                                 trening_krav=True)



# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Umarkert andel
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# CoReg
coreg_resultater = trening_coreg_dag(treningssett_markert,
                                      treningssett_umarkert,
                                      treningssett,
                                      X_test,
                                      y_test,
                                      splitt,
                                      n_runder,
                                      trenings_andel)


# Legger resulater til i datasett
coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_dag

### BHD

In [ ]:
# BHD
bhd_resultater = trening_bhd_dag(treningssett_markert,
                                  treningssett_umarkert,
                                  treningssett,
                                  X_test,
                                  y_test,
                                  splitt,
                                  n_runder,
                                  umarkert_andel=0.25,
                                  trenings_andel=trenings_andel)


# Legger resulater til i datasett
bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_dag

## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.85, 0.9, 0.95, 1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # CoReg
  coreg_resultater = trening_coreg_dag(treningssett_markert,
                                        treningssett_umarkert,
                                        treningssett,
                                        X_test,
                                        y_test,
                                        splitt,
                                        n_runder,
                                        trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_dag

### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # BHD
  bhd_resultater = trening_bhd_dag(treningssett_markert,
                                    treningssett_umarkert,
                                    treningssett,
                                    X_test,
                                    y_test,
                                    splitt,
                                    n_runder,
                                    umarkert_andel=0.25,
                                    trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_dag

## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_design,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  coreg_resultater = trening_coreg_enzym(treningssett_markert,
                                         enzymtyper_treningssett_markert,
                                         treningssett_umarkert,
                                         treningssett,
                                         X_test,
                                         y_test,
                                         splitt=splitt,
                                         gjentagelser=gjentagelser,
                                         n_runder=n_runder,
                                         trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-15 16:24:07,402] A new study created in memory with name: no-name-0c2bb0a8-431e-4333-bf4d-caef6d218a7d
[I 2024-03-15 16:24:47,898] Trial 0 finished with value: 3.8063367883903254 and parameters: {'k1': 9, 'k2': 3, 'p1': 5, 'p2': 3, 'max_iters': 3, 'pool_size': 163}. Best is trial 0 with value: 3.8063367883903254.
[I 2024-03-15 16:25:51,558] Trial 1 finished with value: 3.8001781496954283 and parameters: {'k1': 8, 'k2': 2, 'p1': 7, 'p2': 4, 'max_iters': 10, 'pool_size': 126}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:26:01,396] Trial 2 finished with value: 3.8225169644550157 and parameters: {'k1': 9, 'k2': 3, 'p1': 6, 'p2': 3, 'max_iters': 9, 'pool_size': 18}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:28:23,118] Trial 3 finished with value: 3.84186936894447 and parameters: {'k1': 6, 'k2': 10, 'p1': 2, 'p2': 2, 'max_iters': 11, 'pool_size': 174}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:29:47,396] Trial 4 fi

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,3.806806,0.582990,3.097417,0.159482,2.503015,0.751529,2.052800,0.094294,"{'k1': 9, 'k2': 3, 'p1': 6, 'p2': 1, 'max_iter..."
1,1.0,3.832337,0.577377,3.174067,0.165273,2.124103,0.821947,1.727685,0.081852,"{'k1': 9, 'k2': 2, 'p1': 10, 'p2': 2, 'max_ite..."


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # BHD
  bhd_resultater = trening_bhd_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     umarkert_andel=umarkert_andel,
                                     trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-15 20:17:35,401] A new study created in memory with name: no-name-7213be66-d841-4aa5-ae4c-db1a343129e5
[I 2024-03-15 20:19:27,819] Trial 0 finished with value: 5.246828854906072 and parameters: {'k': 5046, 'alpha': 0.1, 'maks_iterasjoner': 79}. Best is trial 0 with value: 5.246828854906072.
[I 2024-03-15 20:21:01,095] Trial 1 finished with value: 5.246813682909695 and parameters: {'k': 2726, 'alpha': 0.1, 'maks_iterasjoner': 99}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:22:19,322] Trial 2 finished with value: 5.246898540250276 and parameters: {'k': 2368, 'alpha': 0.1, 'maks_iterasjoner': 41}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:24:09,995] Trial 3 finished with value: 8.236193845576382 and parameters: {'k': 4668, 'alpha': 1, 'maks_iterasjoner': 139}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:25:09,395] Trial 4 finished with value: 8.243883651545284 and parameters: {'k': 1153, 'alpha': 1, 'maks_iterasjone

## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_design,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.85, 0.9, 0.95, 1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # CoReg
  coreg_resultater = trening_coreg_dag(treningssett_markert,
                                        treningssett_umarkert,
                                        treningssett,
                                        X_test,
                                        y_test,
                                        splitt,
                                        n_runder,
                                        trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_dag

### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # BHD
  bhd_resultater = trening_bhd_dag(treningssett_markert,
                                    treningssett_umarkert,
                                    treningssett,
                                    X_test,
                                    y_test,
                                    splitt,
                                    n_runder,
                                    umarkert_andel=0.25,
                                    trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_dag

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
collagen_data_uten_NIR = collagen_data.drop(columns=NIR_kolonner)
collagen_data_design_uten_NIR = collagen_data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  coreg_resultater = trening_coreg_enzym(treningssett_markert,
                                         enzymtyper_treningssett_markert,
                                         treningssett_umarkert,
                                         treningssett,
                                         X_test,
                                         y_test,
                                         splitt=splitt,
                                         gjentagelser=gjentagelser,
                                         n_runder=n_runder,
                                         trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-15 16:24:07,402] A new study created in memory with name: no-name-0c2bb0a8-431e-4333-bf4d-caef6d218a7d
[I 2024-03-15 16:24:47,898] Trial 0 finished with value: 3.8063367883903254 and parameters: {'k1': 9, 'k2': 3, 'p1': 5, 'p2': 3, 'max_iters': 3, 'pool_size': 163}. Best is trial 0 with value: 3.8063367883903254.
[I 2024-03-15 16:25:51,558] Trial 1 finished with value: 3.8001781496954283 and parameters: {'k1': 8, 'k2': 2, 'p1': 7, 'p2': 4, 'max_iters': 10, 'pool_size': 126}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:26:01,396] Trial 2 finished with value: 3.8225169644550157 and parameters: {'k1': 9, 'k2': 3, 'p1': 6, 'p2': 3, 'max_iters': 9, 'pool_size': 18}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:28:23,118] Trial 3 finished with value: 3.84186936894447 and parameters: {'k1': 6, 'k2': 10, 'p1': 2, 'p2': 2, 'max_iters': 11, 'pool_size': 174}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:29:47,396] Trial 4 fi

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,3.806806,0.582990,3.097417,0.159482,2.503015,0.751529,2.052800,0.094294,"{'k1': 9, 'k2': 3, 'p1': 6, 'p2': 1, 'max_iter..."
1,1.0,3.832337,0.577377,3.174067,0.165273,2.124103,0.821947,1.727685,0.081852,"{'k1': 9, 'k2': 2, 'p1': 10, 'p2': 2, 'max_ite..."


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # BHD
  bhd_resultater = trening_bhd_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     umarkert_andel=umarkert_andel,
                                     trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-15 20:17:35,401] A new study created in memory with name: no-name-7213be66-d841-4aa5-ae4c-db1a343129e5
[I 2024-03-15 20:19:27,819] Trial 0 finished with value: 5.246828854906072 and parameters: {'k': 5046, 'alpha': 0.1, 'maks_iterasjoner': 79}. Best is trial 0 with value: 5.246828854906072.
[I 2024-03-15 20:21:01,095] Trial 1 finished with value: 5.246813682909695 and parameters: {'k': 2726, 'alpha': 0.1, 'maks_iterasjoner': 99}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:22:19,322] Trial 2 finished with value: 5.246898540250276 and parameters: {'k': 2368, 'alpha': 0.1, 'maks_iterasjoner': 41}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:24:09,995] Trial 3 finished with value: 8.236193845576382 and parameters: {'k': 4668, 'alpha': 1, 'maks_iterasjoner': 139}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:25:09,395] Trial 4 finished with value: 8.243883651545284 and parameters: {'k': 1153, 'alpha': 1, 'maks_iterasjone

## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_uten_NIR,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.85, 0.9, 0.95, 1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # CoReg
  coreg_resultater = trening_coreg_dag(treningssett_markert,
                                        treningssett_umarkert,
                                        treningssett,
                                        X_test,
                                        y_test,
                                        splitt,
                                        n_runder,
                                        trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_dag

### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # BHD
  bhd_resultater = trening_bhd_dag(treningssett_markert,
                                    treningssett_umarkert,
                                    treningssett,
                                    X_test,
                                    y_test,
                                    splitt,
                                    n_runder,
                                    umarkert_andel=0.25,
                                    trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_dag

## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)


# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              collagen_data_design_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  coreg_resultater = trening_coreg_enzym(treningssett_markert,
                                         enzymtyper_treningssett_markert,
                                         treningssett_umarkert,
                                         treningssett,
                                         X_test,
                                         y_test,
                                         splitt=splitt,
                                         gjentagelser=gjentagelser,
                                         n_runder=n_runder,
                                         trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-15 16:24:07,402] A new study created in memory with name: no-name-0c2bb0a8-431e-4333-bf4d-caef6d218a7d
[I 2024-03-15 16:24:47,898] Trial 0 finished with value: 3.8063367883903254 and parameters: {'k1': 9, 'k2': 3, 'p1': 5, 'p2': 3, 'max_iters': 3, 'pool_size': 163}. Best is trial 0 with value: 3.8063367883903254.
[I 2024-03-15 16:25:51,558] Trial 1 finished with value: 3.8001781496954283 and parameters: {'k1': 8, 'k2': 2, 'p1': 7, 'p2': 4, 'max_iters': 10, 'pool_size': 126}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:26:01,396] Trial 2 finished with value: 3.8225169644550157 and parameters: {'k1': 9, 'k2': 3, 'p1': 6, 'p2': 3, 'max_iters': 9, 'pool_size': 18}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:28:23,118] Trial 3 finished with value: 3.84186936894447 and parameters: {'k1': 6, 'k2': 10, 'p1': 2, 'p2': 2, 'max_iters': 11, 'pool_size': 174}. Best is trial 1 with value: 3.8001781496954283.
[I 2024-03-15 16:29:47,396] Trial 4 fi

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.9,3.806806,0.582990,3.097417,0.159482,2.503015,0.751529,2.052800,0.094294,"{'k1': 9, 'k2': 3, 'p1': 6, 'p2': 1, 'max_iter..."
1,1.0,3.832337,0.577377,3.174067,0.165273,2.124103,0.821947,1.727685,0.081852,"{'k1': 9, 'k2': 2, 'p1': 10, 'p2': 2, 'max_ite..."


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # BHD
  bhd_resultater = trening_bhd_enzym(treningssett_markert,
                                     enzymtyper_treningssett_markert,
                                     treningssett_umarkert,
                                     treningssett,
                                     X_test,
                                     y_test,
                                     splitt=splitt,
                                     gjentagelser=gjentagelser,
                                     n_runder=n_runder,
                                     umarkert_andel=umarkert_andel,
                                     trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-15 20:17:35,401] A new study created in memory with name: no-name-7213be66-d841-4aa5-ae4c-db1a343129e5
[I 2024-03-15 20:19:27,819] Trial 0 finished with value: 5.246828854906072 and parameters: {'k': 5046, 'alpha': 0.1, 'maks_iterasjoner': 79}. Best is trial 0 with value: 5.246828854906072.
[I 2024-03-15 20:21:01,095] Trial 1 finished with value: 5.246813682909695 and parameters: {'k': 2726, 'alpha': 0.1, 'maks_iterasjoner': 99}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:22:19,322] Trial 2 finished with value: 5.246898540250276 and parameters: {'k': 2368, 'alpha': 0.1, 'maks_iterasjoner': 41}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:24:09,995] Trial 3 finished with value: 8.236193845576382 and parameters: {'k': 4668, 'alpha': 1, 'maks_iterasjoner': 139}. Best is trial 1 with value: 5.246813682909695.
[I 2024-03-15 20:25:09,395] Trial 4 finished with value: 8.243883651545284 and parameters: {'k': 1153, 'alpha': 1, 'maks_iterasjone

## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                          collagen_data_design_uten_NIR,
                                                          test_andel=test_andel,
                                                          trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.85, 0.9, 0.95, 1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 100

### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # CoReg
  coreg_resultater = trening_coreg_dag(treningssett_markert,
                                        treningssett_umarkert,
                                        treningssett,
                                        X_test,
                                        y_test,
                                        splitt,
                                        n_runder,
                                        trenings_andel)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Data med resultater fra treningsandeler
coreg_resultater_dag

### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # BHD
  bhd_resultater = trening_bhd_dag(treningssett_markert,
                                    treningssett_umarkert,
                                    treningssett,
                                    X_test,
                                    y_test,
                                    splitt,
                                    n_runder,
                                    umarkert_andel=0.25,
                                    trenings_andel=trenings_andel)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Data med resultater fra treningsandeler
bhd_resultater_dag